In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from PIL import Image


HEIGHT, WIDTH, CHANNELS = 250, 250, 3
tf.__version__

'2.6.4'

## Outlook:
### 1. Import, resize and merge X_train and Y_train
### 2. Train model

In [12]:
def createFileList(myDir, format='.jpg'):
    file_names = []
    
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                file_names.append(name)
    return root, file_names

In [13]:
train_X_root, train_X_list = createFileList('../input/ds-fire-detection/train_imgs/train_imgs')  
Y_train_df = pd.read_csv('../input/ds-fire-detection/train_labels.csv',
                      index_col=0,
                      header=None,
                      names=['colums', 'labels'])

../input/ds-fire-detection/train_imgs/train_imgs


In [14]:
def merge_images_labels(root, FileList):
    df = []
    for file in FileList:
        img_file = Image.open(os.path.join(root, file))
        
        img_file = img_file.resize((HEIGHT, WIDTH))
        
        value = np.asarray(img_file.getdata()).flatten()
        
        df.append(np.concatenate((value, Y_train_df.loc[file]),axis=0))
    
    
    return np.array(df)

In [15]:
train = merge_images_labels(train_X_root, train_X_list)

In [16]:
X_train = train[:,:-1].reshape(-1, HEIGHT, WIDTH, CHANNELS)/255
Y_train = train[:,-1].reshape(-1,1)
print(X_train.shape)
print(Y_train.shape)

del train_X_root, train_X_list, train,Y_train_df   # we won't use them

(2619, 250, 250, 3)
(2619, 1)


In [8]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(   # image augmentation
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)   # Note: validation split defined here

datagen.fit(X_train)

## train model

In [ ]:
dropout = 0.25

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 5, activation="relu", padding="same",input_shape=[HEIGHT, WIDTH, CHANNELS]),
    tf.keras.layers.MaxPooling2D(2),

    tf.keras.layers.Conv2D(32, 4, activation="relu"),
    tf.keras.layers.MaxPooling2D(2),

    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(150, activation="relu"),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(150, activation="relu"),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

In [19]:
opt = tf.keras.optimizers.Adam()

# autosave model with best validation accuracy
autosave = tf.keras.callbacks.ModelCheckpoint(
    "./FireDectector.h5",
    monitor='val_accuracy',
    verbose = 1,
    save_best_only=True,
)

# stop training process if model reached specified result
EarlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=20,
    verbose = 1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

# reduce learning rate if validation accuracy doesn't increase
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', 
    patience=8, 
    verbose=1, 
    factor=0.7, 
    min_lr=0.00001)

model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])


In [ ]:
model.fit(datagen.flow(X_train, Y_train, batch_size=32,subset='training'),
         validation_data=datagen.flow(X_train, Y_train, batch_size=8, subset='validation'), 
                    shuffle = True,
                    epochs=50,
                    callbacks = [autosave, EarlyStop, learning_rate_reduction],
                    verbose = 2)
